# Preprocessing of the simulated dataset

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
# change to your path to multimodal_spectroscopic_dataset (to be downloaded from Zenodo, link in ReadME)
path = '../../../data/multimodal_spectroscopic_dataset/'
save_path = '../../../data/sim'

In [2]:
fp_size=128
mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=fp_size)

def get_fingerprint(smiles: str):
    mol = Chem.MolFromSmiles(smiles)
    fp = mfpgen.GetFingerprint(mol)
    return list(fp.ToBitString())

In [5]:
for i in np.arange(0,245):
    df = pd.read_parquet(f'{path}/aligned_chunk_{i}.parquet', columns=['smiles', 'molecular_formula', 'msms_cfmid_positive_10ev', 'msms_cfmid_positive_20ev', 'msms_cfmid_positive_40ev','msms_iceberg_positive','msms_scarf_positive'])
    df = df.rename(columns={'molecular_formula':'formula'})

    data_cfmid_10 = df[['smiles','formula','msms_cfmid_positive_10ev']]
    data_cfmid_10['modality'] = ['msms_cfmid_positive_10ev']*len(data_cfmid_10)
    data_cfmid_10 = data_cfmid_10.rename(columns={'msms_cfmid_positive_10ev':'spectrum'})    
    # data_cfmid_10['coll_en']=['10' for i in np.arange(len(data_cfmid_10))]

    data_cfmid_20 = df[['smiles','formula','msms_cfmid_positive_20ev']]
    data_cfmid_20['modality'] = ['msms_cfmid_positive_20ev']*len(data_cfmid_20)
    data_cfmid_20 = data_cfmid_20.rename(columns={'msms_cfmid_positive_20ev':'spectrum'})    
    # data_cfmid_20['coll_en']=['20' for i in np.arange(len(data_cfmid_20))]

    data_cfmid_40 = df[['smiles','formula','msms_cfmid_positive_40ev']]
    data_cfmid_40['modality'] = ['msms_cfmid_positive_40ev']*len(data_cfmid_40)
    data_cfmid_40 = data_cfmid_40.rename(columns={'msms_cfmid_positive_40ev':'spectrum'})    
    # data_cfmid_40['coll_en']=['40' for i in np.arange(len(data_cfmid_40))]

    data_scarf = df[['smiles','formula','msms_scarf_positive']]
    data_scarf['modality'] = ['msms_scarf_positive']*len(data_scarf)
    data_scarf = data_scarf.rename(columns={'msms_scarf_positive':'spectrum'})
    data_scarf['spectrum'] = data_scarf['spectrum'].apply(lambda row:[s.astype('float64') for s in row])

    data_ice = df[['smiles','formula','msms_iceberg_positive']]
    data_ice['modality'] = ['msms_iceberg_positive']*len(data_ice)
    data_ice = data_ice.rename(columns={'msms_iceberg_positive':'spectrum'})
    data_ice['spectrum'] = data_ice['spectrum'].apply(lambda row:[s.astype('float64') for s in row])

    data_chunk=pd.concat([data_cfmid_10, data_cfmid_20, data_cfmid_40, data_scarf, data_ice], ignore_index=True)
    
    data_chunk['mzs'] = data_chunk.apply(lambda row: [mzs for [mzs, intensity] in row['spectrum']], axis=1)
    data_chunk['intensities'] = data_chunk.apply(lambda row: [intensity for [mzs, intensity] in row['spectrum']], axis=1)

    # calculate molecular fingerprints
    data_chunk['fingerprint'] = data_chunk['smiles'].map(lambda smiles : get_fingerprint(smiles))
    #convert list of strings to list of int
    data_chunk['fingerprint'] = data_chunk['fingerprint'].parallel_apply(lambda row: [int(el) for el in row])
    
    data_chunk[['formula','smiles','spectrum','fingerprint']].to_parquet(f'{save_path}/aligned_chunk_{i}.parquet')

    # data_msms=pd.concat([data_msms, data_chunk], ignore_index=True)
    # data_msms.to_parquet(f'/Users/ism/Projects/mmbart-activeft-project/data/msms/SIM/check-formatting/aligned_chunk_{i}.parquet')
    print('Preprocessed chunk', i)

data_chunk

/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cfmid_10['modality'] = ['msms_cfmid_positive_10ev']*len(data_cfmid_10)
/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cfmid_20['modality'] = ['msms_cfmid_positive_20ev']*len(data_cfmid_20)
/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:16: SettingWithCopyWarning: 
A

Preprocessed chunk 0


/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cfmid_10['modality'] = ['msms_cfmid_positive_10ev']*len(data_cfmid_10)
/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cfmid_20['modality'] = ['msms_cfmid_positive_20ev']*len(data_cfmid_20)
/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:16: SettingWithCopyWarning: 
A

Preprocessed chunk 1


/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cfmid_10['modality'] = ['msms_cfmid_positive_10ev']*len(data_cfmid_10)
/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cfmid_20['modality'] = ['msms_cfmid_positive_20ev']*len(data_cfmid_20)
/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:16: SettingWithCopyWarning: 
A

...

/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cfmid_10['modality'] = ['msms_cfmid_positive_10ev']*len(data_cfmid_10)
/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cfmid_20['modality'] = ['msms_cfmid_positive_20ev']*len(data_cfmid_20)
/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:16: SettingWithCopyWarning: 
A

Preprocessed chunk 243


/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cfmid_10['modality'] = ['msms_cfmid_positive_10ev']*len(data_cfmid_10)
/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cfmid_20['modality'] = ['msms_cfmid_positive_20ev']*len(data_cfmid_20)
/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_88285/71491734.py:16: SettingWithCopyWarning: 
A

Preprocessed chunk 244


,smiles,formula,spectrum,modality,mzs,intensities,fingerprint
0,O=C(O)c1ccnc2c(Br)cccc12,C10H6BrNO2,"[[233.9549, 100.0], [251.96547, 77.68]]",msms_cfmid_positive_10ev,"[233.9549, 251.96547]","[100.0, 77.68]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,CN(N(C(=O)c1ccccc1)c1ccccn1)C(C)(C)C,C17H21N3O,"[[228.11314, 32.79], [284.17574, 100.0]]",msms_cfmid_positive_10ev,"[228.11314, 284.17574]","[32.79, 100.0]","[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, ..."
2,CN(C)S(=O)(=O)n1ccnc1CNCc1ccccc1,C13H18N4O2S,"[[250.06447, 10.63], [295.12232, 100.0]]",msms_cfmid_positive_10ev,"[250.06447, 295.12232]","[10.63, 100.0]","[1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, ..."
3,OCc1cc2cccnn2c1-c1ccccn1,C13H11N3O,"[[208.08692, 29.16], [226.09749, 100.0]]",msms_cfmid_positive_10ev,"[208.08692, 226.09749]","[29.16, 100.0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
4,CC1(C)OC(=O)N(c2ccc(C(=O)NNc3ccc(Cl)nn3)cc2)[C...,C22H20ClN5O3,"[[420.12218, 7.44], [438.13274, 100.0]]",msms_cfmid_positive_10ev,"[420.12218, 438.13274]","[7.44, 100.0]","[1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...
16305,NC12CC3CC(CC(C3)C1O)C2,C10H17NO,"[[25.00727653503418, 0.06718017160892487], [26...",msms_iceberg_positive,"[25.00727653503418, 26.01510238647461, 27.0103...","[0.06718017160892487, 0.14771266281604767, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
16306,COc1nc(SC)ncc1C(=O)Cl,C7H7ClN2O2S,"[[25.00727653503418, 0.1762629747390747], [26....",msms_iceberg_positive,"[25.00727653503418, 26.01510238647461, 27.0229...","[0.1762629747390747, 0.236970454454422, 0.1581...","[1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, ..."
16307,CCCNc1ccc(Br)cc1C(=O)OC,C11H14BrNO2,"[[29.038576126098633, 0.2817704975605011], [30...",msms_iceberg_positive,"[29.038576126098633, 30.046401977539062, 31.01...","[0.2817704975605011, 0.3150026500225067, 0.028...","[1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, ..."
16308,NNCc1ccc(Cl)c(C(F)(F)F)c1,C8H8ClF3N2,"[[25.00727653503418, 0.3358204662799835], [26....",msms_iceberg_positive,"[25.00727653503418, 26.01510238647461, 27.0229...","[0.3358204662799835, 0.11234257370233536, 0.26...","[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, ..."


In [6]:
data_chunk

,smiles,formula,spectrum,modality,mzs,intensities,fingerprint
0,O=C(O)c1ccnc2c(Br)cccc12,C10H6BrNO2,"[[233.9549, 100.0], [251.96547, 77.68]]",msms_cfmid_positive_10ev,"[233.9549, 251.96547]","[100.0, 77.68]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,CN(N(C(=O)c1ccccc1)c1ccccn1)C(C)(C)C,C17H21N3O,"[[228.11314, 32.79], [284.17574, 100.0]]",msms_cfmid_positive_10ev,"[228.11314, 284.17574]","[32.79, 100.0]","[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, ..."
2,CN(C)S(=O)(=O)n1ccnc1CNCc1ccccc1,C13H18N4O2S,"[[250.06447, 10.63], [295.12232, 100.0]]",msms_cfmid_positive_10ev,"[250.06447, 295.12232]","[10.63, 100.0]","[1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, ..."
3,OCc1cc2cccnn2c1-c1ccccn1,C13H11N3O,"[[208.08692, 29.16], [226.09749, 100.0]]",msms_cfmid_positive_10ev,"[208.08692, 226.09749]","[29.16, 100.0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
4,CC1(C)OC(=O)N(c2ccc(C(=O)NNc3ccc(Cl)nn3)cc2)[C...,C22H20ClN5O3,"[[420.12218, 7.44], [438.13274, 100.0]]",msms_cfmid_positive_10ev,"[420.12218, 438.13274]","[7.44, 100.0]","[1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...
16305,NC12CC3CC(CC(C3)C1O)C2,C10H17NO,"[[25.00727653503418, 0.06718017160892487], [26...",msms_iceberg_positive,"[25.00727653503418, 26.01510238647461, 27.0103...","[0.06718017160892487, 0.14771266281604767, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
16306,COc1nc(SC)ncc1C(=O)Cl,C7H7ClN2O2S,"[[25.00727653503418, 0.1762629747390747], [26....",msms_iceberg_positive,"[25.00727653503418, 26.01510238647461, 27.0229...","[0.1762629747390747, 0.236970454454422, 0.1581...","[1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, ..."
16307,CCCNc1ccc(Br)cc1C(=O)OC,C11H14BrNO2,"[[29.038576126098633, 0.2817704975605011], [30...",msms_iceberg_positive,"[29.038576126098633, 30.046401977539062, 31.01...","[0.2817704975605011, 0.3150026500225067, 0.028...","[1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, ..."
16308,NNCc1ccc(Cl)c(C(F)(F)F)c1,C8H8ClF3N2,"[[25.00727653503418, 0.3358204662799835], [26....",msms_iceberg_positive,"[25.00727653503418, 26.01510238647461, 27.0229...","[0.3358204662799835, 0.11234257370233536, 0.26...","[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, ..."


In [ ]:
# check
df = pd.read_parquet(f'{save_path}/aligned_chunk_{i}.parquet')
df

,formula,smiles,spectrum,fingerprint
0,C10H6BrNO2,O=C(O)c1ccnc2c(Br)cccc12,"[[233.9549, 100.0], [251.96547, 77.68]]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,C17H21N3O,CN(N(C(=O)c1ccccc1)c1ccccn1)C(C)(C)C,"[[228.11314, 32.79], [284.17574, 100.0]]","[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, ..."
2,C13H18N4O2S,CN(C)S(=O)(=O)n1ccnc1CNCc1ccccc1,"[[250.06447, 10.63], [295.12232, 100.0]]","[1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, ..."
3,C13H11N3O,OCc1cc2cccnn2c1-c1ccccn1,"[[208.08692, 29.16], [226.09749, 100.0]]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
4,C22H20ClN5O3,CC1(C)OC(=O)N(c2ccc(C(=O)NNc3ccc(Cl)nn3)cc2)[C...,"[[420.12218, 7.44], [438.13274, 100.0]]","[1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, ..."
...,...,...,...,...
16305,C10H17NO,NC12CC3CC(CC(C3)C1O)C2,"[[25.00727653503418, 0.06718017160892487], [26...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
16306,C7H7ClN2O2S,COc1nc(SC)ncc1C(=O)Cl,"[[25.00727653503418, 0.1762629747390747], [26....","[1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, ..."
16307,C11H14BrNO2,CCCNc1ccc(Br)cc1C(=O)OC,"[[29.038576126098633, 0.2817704975605011], [30...","[1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, ..."
16308,C8H8ClF3N2,NNCc1ccc(Cl)c(C(F)(F)F)c1,"[[25.00727653503418, 0.3358204662799835], [26....","[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, ..."
